In [1]:
from IPython.display import clear_output
!pip install imutils
clear_output()

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import scipy
!pip install colorama
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from tqdm import tqdm, tqdm_notebook
from colorama import Fore
import json
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *
import time
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import datetime

import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import *
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard
from keras.losses import *

init_notebook_mode(connected=True)
RANDOM_SEED = 42

In [5]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nikhilis007","key":"58b4620045800c66c80eceed85e45354"}'}

In [6]:
!mkdir ~/.kaggle

In [7]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d akash2907/bird-species-classification

 98% 1.35G/1.37G [00:10<00:00, 218MB/s]
100% 1.37G/1.37G [00:10<00:00, 140MB/s]


In [10]:
!unzip /content/bird-species-classification.zip

Archive:  /content/bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: t

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255) 

In [12]:
train_path = "/content/train_data/train_data"
test_path = "/content/test_data/test_data"

In [13]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


Using custom CNN model

In [14]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

In [15]:
num_classes = 16

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [16]:
model.fit_generator(train,validation_data=test,epochs=20,steps_per_epoch=len(train),validation_steps=len(test))

Epoch 1/20
7/7 [==============================] - 103s 14s/step - loss: 2.7357 - accuracy: 0.1533 - val_loss: 2.7151 - val_accuracy: 0.1529
Epoch 2/20
7/7 [==============================] - 84s 13s/step - loss: 2.5267 - accuracy: 0.1667 - val_loss: 2.9391 - val_accuracy: 0.0764
Epoch 3/20
7/7 [==============================] - 119s 19s/step - loss: 2.5674 - accuracy: 0.1800 - val_loss: 2.7297 - val_accuracy: 0.1338
Epoch 4/20
7/7 [==============================] - 84s 13s/step - loss: 2.5266 - accuracy: 0.1667 - val_loss: 2.8281 - val_accuracy: 0.1210
Epoch 5/20
7/7 [==============================] - 85s 13s/step - loss: 2.3253 - accuracy: 0.2400 - val_loss: 2.8435 - val_accuracy: 0.1019
Epoch 6/20
7/7 [==============================] - 82s 13s/step - loss: 2.1490 - accuracy: 0.2733 - val_loss: 2.8107 - val_accuracy: 0.2293
Epoch 7/20
7/7 [==============================] - 84s 13s/step - loss: 1.9733 - accuracy: 0.3200 - val_loss: 3.0676 - val_accuracy: 0.1720
Epoch 8/20
7/7 [=========

Using transfer learning

In [17]:
from tensorflow.keras.applications import VGG16, VGG19
base_model= VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))

80134624/80134624 [==============================] - 0s 0us/step


In [18]:
for layer in base_model.layers[:19]:
    layer.trainable = False

model1 = Sequential([
    base_model,
    MaxPool2D((2,2) , strides = 2),
    Flatten(),
    Dense(16 , activation='softmax')])

model1.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy','AUC'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 3, 3, 512)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 16)                73744     
                                                                 
Total params: 20,098,128
Trainable params: 4,793,360
Non-trainable params: 15,304,768
_________________________________________________________________


In [19]:
model1.fit_generator(train,validation_data=test,epochs=20,steps_per_epoch=len(train),validation_steps=len(test))

Epoch 1/20
7/7 [==============================] - 94s 14s/step - loss: 2.6124 - accuracy: 0.2200 - auc: 0.6998 - val_loss: 2.6246 - val_accuracy: 0.2675 - val_auc: 0.7337
Epoch 2/20
7/7 [==============================] - 86s 14s/step - loss: 1.4866 - accuracy: 0.5333 - auc: 0.9172 - val_loss: 2.9168 - val_accuracy: 0.3567 - val_auc: 0.7464
Epoch 3/20
7/7 [==============================] - 84s 13s/step - loss: 1.0266 - accuracy: 0.7067 - auc: 0.9598 - val_loss: 3.1047 - val_accuracy: 0.3376 - val_auc: 0.7702
Epoch 4/20
7/7 [==============================] - 82s 13s/step - loss: 0.5692 - accuracy: 0.8400 - auc: 0.9833 - val_loss: 3.3489 - val_accuracy: 0.3694 - val_auc: 0.7502
Epoch 5/20
7/7 [==============================] - 83s 13s/step - loss: 0.3525 - accuracy: 0.8800 - auc: 0.9927 - val_loss: 3.8415 - val_accuracy: 0.3503 - val_auc: 0.7631
Epoch 6/20
7/7 [==============================] - 83s 13s/step - loss: 0.1940 - accuracy: 0.9533 - auc: 0.9951 - val_loss: 4.4809 - val_accuracy: